In [36]:
from datasets import load_from_disk
import pandas as pd
import os

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)

# Arrow 파일 경로 설정
train_file_path = os.path.join(parent_dir, "data", "train_dataset")
test_file_path = os.path.join(parent_dir, "data", "test_dataset")
dataset = load_from_disk(train_file_path)
# test_dataset = load_from_disk(test_file_path)

In [37]:
train_dataset = dataset["train"]
valid_dataset = dataset["validation"]
# test_dataset = test_dataset["validation"]

train_df = pd.DataFrame(train_dataset)
valid_df = pd.DataFrame(valid_dataset)
# test_df = pd.DataFrame(test_dataset)

In [38]:
import re
import ast


def preprocess(text):
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"\\n", " ", text)
    text = re.sub(r"#", " ", text)
    text = re.sub(
        r"[^A-Za-z0-9가-힣.?!,()~‘’“”" ":%&《》〈〉''㈜·\-'+\s一-龥サマーン]",
        "",
        text,
    )  # サマーン 는 predictions.json에 있었음
    text = re.sub(r"\s+", " ", text).strip()  # 두 개 이상의 연속된 공백을 하나로 치환
    # text = re.sub(r"[^A-Za-z0-9가-힣.?!,()~‘’“”"":%&《》〈〉''㈜·\-\'+\s一-龥]", "", text)

    return text

# rerank 결과에 타이틀 추가

In [39]:
import json
import pandas as pd
import os

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
file_path = os.path.join(parent_dir, "data", "wikipedia_documents.json")

with open(file_path, "r", encoding="utf-8") as json_file:
    json_data = json.load(json_file)

df_wiki = pd.read_json(file_path).transpose()

In [41]:
df_wiki["text"] = df_wiki["text"].apply(preprocess)

In [43]:
# es_path = os.path.join(parent_dir, "data", "es_train")
# df_train_reranked = pd.read_csv(os.path.join(es_path, "not_test_es_40_rerank_5.csv"))

# df_train_reranked["context"] = " ".join(
#     [
#         [
#             f"{df_wiki[df_wiki['text']==c]['title']}: {c}"
#             for c in row["context"]
#         ]
#         for _, row in df_train_reranked.iterrows()
#     ]
# )
# df_train_reranked.to_csv("test_es_40_rerank_5_title.csv", index=False)

In [50]:
es_path = os.path.join(parent_dir, "data", "es_train")
df_train_reranked = pd.read_csv(os.path.join(es_path, "not_test_es_40_rerank_5.csv"))

df_train_reranked["context"] = df_train_reranked["context"].apply(
    lambda x: ast.literal_eval(x)
)

In [ ]:
def construct_context(row, wiki_df):
    return " ".join(
        [
            f"{wiki_df.loc[wiki_df['text'] == c, 'title'].values[0]}: {c}"
            for c in row
            if not wiki_df.loc[wiki_df["text"] == c, "title"].empty
        ]
    )

df_train_reranked["context"] = df_train_reranked["context"].apply(
    construct_context, wiki_df=df_wiki
)

# context에 negative passage 추가된 데이터셋 만들기

In [105]:
# 정답 검증 함수
def verify_answer(row):
    context = row["context"]
    answer_text = row["answers"]["text"][0]
    answer_start = row["answers"]["answer_start"][0]

    # answer_start부터 answer_text 길이만큼의 문장을 추출
    extracted_answer = context[answer_start : answer_start + len(answer_text)]

    # 추출한 정답과 answer_text가 동일한지 검증
    return extracted_answer == answer_text


# 잘못된 정답을 포함한 행만 모아서 새로운 DataFrame 반환
def get_incorrect_rows(df):
    # 각 행에 대해 검증 수행하여 False인 행만 필터링
    incorrect_df = df[~df.apply(verify_answer, axis=1)]
    return incorrect_df

# 정답 찾는 함수
def find_start_position(row):
    context = row["context"]
    answer = row["answers"]
    start_position = context.find(answer["text"][0])
    answer["answer_start"][0] = start_position
    return answer

### Validation

In [148]:
# val 
es_path = os.path.join(parent_dir, "data", "es_train")
df_train_reranked = pd.read_csv(os.path.join(es_path, "not_train_es_100_rerank_5.csv"))

In [149]:
df_train_reranked["context"] = df_train_reranked["context"].apply(
    lambda x: ast.literal_eval(x)
)
df_train_reranked["answers"] = df_train_reranked["answers"].apply(
    lambda x: ast.literal_eval(x)
)

In [150]:
df_train_reranked["context"] = [
    (
        row["context"] + [preprocess(row["original_context"])]
        if preprocess(row["original_context"]) not in row["context"]
        else row["context"]
    )
    for _, row in df_train_reranked.iterrows()
]

In [151]:
df_train_reranked["context"] = df_train_reranked["context"].apply(lambda x: " ".join(x))
df_train_reranked["answers"] = df_train_reranked.apply(find_start_position, axis=1)

In [152]:
df_train_reranked.head()

,question,id,context,original_context,answers
0,처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?,mrc-0-003264,"순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법...","순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법...","{'answer_start': [192], 'text': ['한보철강']}"
1,스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장된 연도는?,mrc-0-004762,요크 카운티 동쪽에 처음으로 여객 열차 운행이 시작한 시점은 1868년 토론토 & ...,요크 카운티 동쪽에 처음으로 여객 열차 운행이 시작한 시점은 1868년 토론토 & ...,"{'answer_start': [108], 'text': ['1871년']}"
2,촌락에서 운영 위원 후보자 이름을 쓰기위해 사용된 것은?,mrc-1-001810,"촐라 정부 촐라의 정부 체제는 전제군주제였으며,2001 촐라의 군주는 절대적인 권력...","촐라 정부\n 촐라의 정부 체제는 전제군주제였으며,2001 촐라의 군주는 절대적인 ...","{'answer_start': [509], 'text': ['나뭇잎']}"
3,로타이르가 백조를 구하기 위해 사용한 것은?,mrc-1-000219,프랑스의 십자군 무훈시는 1099년 예루살렘 왕국의 통치자가 된 고드프루아 드 부용...,프랑스의 십자군 무훈시는 1099년 예루살렘 왕국의 통치자가 된 고드프루아 드 부용...,"{'answer_start': [1100], 'text': ['금대야']}"
4,의견을 자유롭게 나누는 것은 조직 내 어떤 관계에서 가능한가?,mrc-1-000285,탈관료제화는 현대사회에서 관료제 성격이 약화되는 현상이다. 현대사회에서 관료제는 약...,탈관료제화는 현대사회에서 관료제 성격이 약화되는 현상이다. 현대사회에서 관료제는 약...,"{'answer_start': [383], 'text': ['수평적 관계']}"


In [130]:
def update_answer_start(row):
    answers = row["answers"]
    context = row["context"]

    # answer_start가 -1인 경우에만 전처리 수행
    if answers["answer_start"][0] == -1:
        # text 전처리
        original_text = answers["text"][0]
        processed_text = preprocess(original_text)

        # 전처리된 텍스트로 시작 위치 다시 찾기
        new_start = context.find(processed_text)

        # 새로운 딕셔너리 반환
        return {"text": [processed_text], "answer_start": [new_start]}

    return answers


# answer_start가 -1인 행만 업데이트
mask = df_train_reranked["answers"].apply(lambda x: x["answer_start"][0] == -1)
df_train_reranked.loc[mask, "answers"] = df_train_reranked[mask].apply(
    update_answer_start, axis=1
)

In [131]:
df_train_reranked["answers"] = df_train_reranked.apply(find_start_position, axis=1)

In [132]:
# answer_start가 [-1]인 행 찾기
invalid_rows = df_train_reranked[
    df_train_reranked["answers"].apply(lambda x: x["answer_start"][0] == -1)
]
invalid_rows

,question,id,context,original_context,answers


In [154]:
df_train_reranked.to_csv("n_passage_train_validation_dataset.csv", index=False)

### train

In [134]:
# train
es_path = os.path.join(parent_dir, "data", "es_train")
df_train_reranked = pd.read_csv(
    os.path.join(es_path, "train_es_train_100_rerank_5_for_dataset.csv")
)

In [135]:
df_train_reranked["context"] = [
    (
        row["context"] + preprocess(row["original_context"])
        if preprocess(row["original_context"]) not in row["context"]
        else row["context"]
    )
    for _, row in df_train_reranked.iterrows()
]

In [136]:
df_train_reranked.head()

,question,id,context,original_context,answers
0,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,"미국의 외교정책의 수립과 이행에 대한 주된 책임을 대통령이 지게 되어 있으나, 의회...",미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,"{'answer_start': [235], 'text': ['하원']}"
1,현대적 인사조직관리의 시발점이 된 책은?,mrc-0-004397,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,"{'answer_start': [212], 'text': ['《경영의 실제》']}"
2,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,mrc-1-000362,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,"{'answer_start': [510], 'text': ['백성']}"
3,11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,mrc-0-001510,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...","불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...","{'answer_start': [625], 'text': ['중국']}"
4,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,mrc-0-000823,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,"{'answer_start': [30], 'text': ['4개']}"


In [137]:
df_train_reranked["answers"] = df_train_reranked["answers"].apply(
    lambda x: ast.literal_eval(x)
)

In [138]:
df_train_reranked["answers"] = df_train_reranked.apply(find_start_position, axis=1)

In [139]:
df_train_reranked.head()

,question,id,context,original_context,answers
0,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,"미국의 외교정책의 수립과 이행에 대한 주된 책임을 대통령이 지게 되어 있으나, 의회...",미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,"{'answer_start': [100], 'text': ['하원']}"
1,현대적 인사조직관리의 시발점이 된 책은?,mrc-0-004397,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,"{'answer_start': [212], 'text': ['《경영의 실제》']}"
2,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,mrc-1-000362,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,"{'answer_start': [362], 'text': ['백성']}"
3,11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,mrc-0-001510,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...","불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...","{'answer_start': [615], 'text': ['중국']}"
4,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,mrc-0-000823,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,"{'answer_start': [30], 'text': ['4개']}"


In [140]:
def update_answer_start(row):
    answers = row["answers"]
    context = row["context"]

    # answer_start가 -1인 경우에만 전처리 수행
    if answers["answer_start"][0] == -1:
        # text 전처리
        original_text = answers["text"][0]
        processed_text = preprocess(original_text)

        # 전처리된 텍스트로 시작 위치 다시 찾기
        new_start = context.find(processed_text)

        # 새로운 딕셔너리 반환
        return {"text": [processed_text], "answer_start": [new_start]}

    return answers


# answer_start가 -1인 행만 업데이트
mask = df_train_reranked["answers"].apply(lambda x: x["answer_start"][0] == -1)
df_train_reranked.loc[mask, "answers"] = df_train_reranked[mask].apply(
    update_answer_start, axis=1
)

In [141]:
df_train_reranked["answers"] = df_train_reranked.apply(find_start_position, axis=1)

In [142]:
# answer_start가 [-1]인 행 찾기
invalid_rows = df_train_reranked[
    df_train_reranked["answers"].apply(lambda x: x["answer_start"][0] == -1)
]
invalid_rows

,question,id,context,original_context,answers


In [147]:
df_train_reranked

,question,id,context,original_context,answers
0,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,"미국의 외교정책의 수립과 이행에 대한 주된 책임을 대통령이 지게 되어 있으나, 의회...",미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,"{'answer_start': [100], 'text': ['하원']}"
1,현대적 인사조직관리의 시발점이 된 책은?,mrc-0-004397,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,"{'answer_start': [212], 'text': ['《경영의 실제》']}"
2,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,mrc-1-000362,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,"{'answer_start': [362], 'text': ['백성']}"
3,11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,mrc-0-001510,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...","불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...","{'answer_start': [615], 'text': ['중국']}"
4,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,mrc-0-000823,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,"{'answer_start': [30], 'text': ['4개']}"
...,...,...,...,...,...
3947,대부분의 이오의 산이 형성되는데 영향을 끼친 것은?,mrc-0-005285,이오의 산 이오의 산 목록 이오에는 100~150개의 산이 있다. 이들 산의 평균 ...,이오의 산\n이오의 산 목록\n 이오에는 100~150개의 산이 있다. 이들 산의 ...,"{'answer_start': [309], 'text': ['지질 구조']}"
3948,애니는 어디서 태어났는가?,mrc-1-000699,애니 점프 캐논은 1863년 12월 11일 델라웨어 주 도버에서 태어났다. 그녀는 ...,애니의 고군분투 뉴욕 입성기!!\n\n인류학자가 꿈인 21살 소녀 '애니(스칼렛 요...,"{'answer_start': [1098], 'text': ['뉴저지']}"
3949,1854년에 니콜라이의 뒤를 이어 상임 지휘자로서 활동한 인물은?,mrc-0-003429,1842년에 작곡가이자 지휘자인 오토 니콜라이가 빈 궁정 오페라극장 소속 관현악단을...,1842년에 작곡가이자 지휘자인 오토 니콜라이가 빈 궁정 오페라극장 소속 관현악단을...,"{'answer_start': [252], 'text': ['칼 에케르트']}"
3950,드래곤의 암수 구분이 있는 신화는?,mrc-0-003956,원어는 고대 그리스어까지 거슬러 올라간다. 영어 문헌에 이 말이 나타나기 시작한 것...,원어는 고대 그리스어까지 거슬러 올라간다. 영어 문헌에 이 말이 나타나기 시작한 것...,"{'answer_start': [452], 'text': ['슬라브 신화']}"


In [143]:
df_train_reranked.to_csv("n_passage_train_dataset.csv", index=False)